In [ ]:
from google import genai
from google.genai import types
import time




In [ ]:
client = genai.Client()



In [ ]:
# ---------------------------------------------------------
# 1. Upload multiple files (these can be ANY files you want)
# ---------------------------------------------------------
businessfile = client.files.upload(file='businesszerotoone.pdf', config={'name':'businessbook'})
sciencefile = client.files.upload(file='sciencebook.pdf', config={'name': 'sciencebook'})
storyfile = client.files.upload(file='storyharry.pdf', config={'name': 'storyharry'})



In [ ]:
# ---------------------------------------------------------
# 2. Create TWO File Search Stores
# ---------------------------------------------------------
businessstore = client.file_search_stores.create(
    config={'display_name': 'businessstore'}
)

sciencestore = client.file_search_stores.create(
    config={'display_name': 'sciencestore'}
)
storystore = client.file_search_stores.create(
    config={'display_name': 'storystore'}
)


In [ ]:
# ---------------------------------------------------------
# 3. Import file1 into Store A
# ---------------------------------------------------------
op1 = client.file_search_stores.import_file(
    file_search_store_name=businessstore.name,
    file_name=businessfile.name
)

# 4. Import file2 into Store B
op2 = client.file_search_stores.import_file(
    file_search_store_name=sciencestore.name,
    file_name=sciencefile.name
)

op3 = client.file_search_stores.import_file(
    file_search_store_name=storystore.name,
    file_name=storyfile.name
)


In [ ]:
# ---------------------------------------------------------
# 5. Wait for both imports to complete
# ---------------------------------------------------------
while not op1.done or not op2.done:
    time.sleep(5)
    if not op1.done:
        op1 = client.operations.get(op1)
    if not op2.done:
        op2 = client.operations.get(op2)
    if not op3.done:
        op3 = client.operations.get(op3)

